#  Sequence to Sequence Learning
:label:`sec_seq2seq`

###### NOTE: Currently this is still Work in Progress (WIP) as the loss result isn't the same as the one in d2l.ai book. The team will be taking a look at this and updating it whenever possible.

As we have seen in :numref:`sec_machine_translation`,
in machine translation
both the input and output are a variable-length sequence.
To address this type of problem,
we have designed a general encoder-decoder architecture
in :numref:`sec_encoder-decoder`.
In this section,
we will
use two RNNs to design
the encoder and the decoder of
this architecture
and apply it to *sequence to sequence* learning
for machine translation
:cite:`Sutskever.Vinyals.Le.2014,Cho.Van-Merrienboer.Gulcehre.ea.2014`.

Following the design principle
of the encoder-decoder architecture,
the RNN encoder can
take a variable-length sequence as the input and transforms it into a fixed-shape hidden state.
In other words,
information of the input (source) sequence
is *encoded* in the hidden state of the RNN encoder.
To generate the output sequence token by token,
a separate RNN decoder
can predict the next token based on
what tokens have been seen (such as in language modeling) or generated,
together with the encoded information of the input sequence.
:numref:`fig_seq2seq` illustrates
how to use two RNNs
for sequence to sequence learning
in machine translation.


![Sequence to sequence learning with an RNN encoder and an RNN decoder.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/seq2seq.svg)
:label:`fig_seq2seq`

In :numref:`fig_seq2seq`,
the special "&lt;eos&gt;" token
marks the end of the sequence.
The model can stop making predictions
once this token is generated.
At the initial time step of the RNN decoder,
there are two special design decisions.
First, the special beginning-of-sequence "&lt;bos&gt;" token is an input.
Second,
the final hidden state of the RNN encoder is used
to initiate the hidden state of the decoder.
In designs such as :cite:`Sutskever.Vinyals.Le.2014`,
this is exactly
how the encoded input sequence information
is fed into the decoder for generating the output (target) sequence.
In some other designs such as :cite:`Cho.Van-Merrienboer.Gulcehre.ea.2014`,
the final hidden state of the encoder
is also fed into the decoder as
part of the inputs
at every time step as shown in :numref:`fig_seq2seq`.
Similar to the training of language models in
:numref:`sec_language_model`,
we can allow the labels to be the original output sequence,
shifted by one token:
"&lt;bos&gt;", "Ils", "regardent", "." $\rightarrow$
"Ils", "regardent", ".", "&lt;eos&gt;".


In the following,
we will explain the design of :numref:`fig_seq2seq`
in greater detail.
We will train this model for machine translation
on the English-French dataset as introduced in
:numref:`sec_machine_translation`.


In [52]:
%use @file[../djl-pytorch.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
import jp.live.ugai.d2j.timemachine.RNNModelScratch
import jp.live.ugai.d2j.timemachine.TimeMachine.trainCh8
import jp.live.ugai.d2j.timemachine.TimeMachineDataset
import jp.live.ugai.d2j.timemachine.Vocab
import jp.live.ugai.d2j.RNNModel
import jp.live.ugai.d2j.util.StopWatch
import jp.live.ugai.d2j.util.Accumulator
import jp.live.ugai.d2j.util.Training
import jp.live.ugai.d2j.util.TrainingChapter9
import jp.live.ugai.d2j.lstm.Decoder
import jp.live.ugai.d2j.lstm.Encoder
import jp.live.ugai.d2j.lstm.EncoderDecoder
import jp.live.ugai.d2j.util.NMT
import java.util.Locale
import kotlin.random.Random
import kotlin.collections.List
import kotlin.collections.Map
import kotlin.Pair
//%load ../utils/djl-imports
//%load ../utils/plot-utils
//%load ../utils/Functions.java
//%load ../utils/PlotUtils.java

//%load ../utils/StopWatch.java
//%load ../utils/Accumulator.java
//%load ../utils/Animator.java
//%load ../utils/TrainingChapter9.java
//%load ../utils/timemachine/Vocab.java
//%load ../utils/timemachine/RNNModel.java
//%load ../utils/timemachine/RNNModelScratch.java
//%load ../utils/timemachine/TimeMachine.java
//%load ../utils/timemachine/TimeMachineDataset.java
//%load ../utils/NMT.java
//%load ../utils/lstm/Encoder.java
//%load ../utils/lstm/Decoder.java
//%load ../utils/lstm/EncoderDecoder.java

In [53]:
// import java.util.stream.*;
// import ai.djl.modality.nlp.*;
// import ai.djl.modality.nlp.embedding.*;
import ai.djl.modality.nlp.DefaultVocabulary
import ai.djl.modality.nlp.Vocabulary
import ai.djl.modality.nlp.embedding.TrainableWordEmbedding

In [54]:
val manager = NDManager.newBaseManager()
val ps = ParameterStore(manager, false)

## Encoder

Technically speaking,
the encoder transforms an input sequence of variable length into a fixed-shape *context variable* $\mathbf{c}$, and encodes the input sequence information in this context variable.
As depicted in :numref:`fig_seq2seq`,
we can use an RNN to design the encoder.

Let us consider a sequence example (batch size: 1).
Suppose that
the input sequence is $x_1, \ldots, x_T$, such that $x_t$ is the $t^{\mathrm{th}}$ token in the input text sequence.
At time step $t$, the RNN transforms
the input feature vector $\mathbf{x}_t$ for $x_t$
and the hidden state $\mathbf{h} _{t-1}$ from the previous time step
into the current hidden state $\mathbf{h}_t$.
We can use a function $f$ to express the transformation of the RNN's recurrent layer:

$$\mathbf{h}_t = f(\mathbf{x}_t, \mathbf{h}_{t-1}). $$

In general,
the encoder transforms the hidden states at
all the time steps
into the context variable through a customized function $q$:

$$\mathbf{c} =  q(\mathbf{h}_1, \ldots, \mathbf{h}_T).$$

For example, when choosing $q(\mathbf{h}_1, \ldots, \mathbf{h}_T) = \mathbf{h}_T$ such as in :numref:`fig_seq2seq`,
the context variable is just the hidden state $\mathbf{h}_T$
of the input sequence at the final time step.

So far we have used a unidirectional RNN
to design the encoder,
where
a hidden state only depends on
the input subsequence at and before the time step of the hidden state.
We can also construct encoders using bidirectional RNNs. In this case, a hidden state depends on
the subsequence before and after the time step (including the input at the current time step), which encodes the information of the entire sequence.


Now let us implement the RNN encoder.
Note that we use an *embedding layer*
to obtain the feature vector for each token in the input sequence.
The weight
of an embedding layer
is a matrix
whose number of rows equals to the size of the input vocabulary (`vocabSize`)
and number of columns equals to the feature vector's dimension (`embedSize`).
For any input token index $i$,
the embedding layer
fetches the $i^{\mathrm{th}}$ row (starting from 0) of the weight matrix
to return its feature vector.
Besides,
here we choose a multilayer GRU to
implement the encoder.


In [55]:
class Seq2SeqEncoder(vocabSize: Int, embedSize: Int, numHiddens: Int, numLayers: Int, dropout: Float) : Encoder() {
    private val embedding: TrainableWordEmbedding
    private val rnn: GRU

    /* The RNN encoder for sequence to sequence learning. */
    init {
        val list: List<String> = (0 until vocabSize).map { it.toString() }
        val vocab: Vocabulary = DefaultVocabulary(list)
        // Embedding layer
        embedding = TrainableWordEmbedding.builder()
            .optNumEmbeddings(vocabSize)
            .setEmbeddingSize(embedSize)
            .setVocabulary(vocab)
            .build()
        addChildBlock("embedding", embedding)
        rnn = GRU.builder()
            .setNumLayers(numLayers)
            .setStateSize(numHiddens)
            .optReturnState(true)
            .optBatchFirst(false)
            .optDropRate(dropout)
            .build()
        addChildBlock("rnn", rnn)
    }

    /** {@inheritDoc}  */
    override fun initializeChildBlocks(manager: NDManager, dataType: DataType, vararg inputShapes: Shape) {
        embedding.initialize(manager, dataType, inputShapes[0])
        val shapes: Array<Shape> = embedding.getOutputShapes(arrayOf(inputShapes[0]))
        manager.newSubManager().use { sub ->
            var nd = sub.zeros(shapes[0], dataType)
            nd = nd.swapAxes(0, 1)
            rnn.initialize(manager, dataType, nd.shape)
        }
    }

    override fun getOutputShapes(inputShapes: Array<Shape>): Array<Shape> {
        val shapes: Array<Shape> = embedding.getOutputShapes(arrayOf(inputShapes[0]))
        val embedShape = shapes[0]
        val swapped = Shape(embedShape[1], embedShape[0], embedShape[2])
        return rnn.getOutputShapes(arrayOf(swapped))
    }


    override fun forwardInternal(
        ps: ParameterStore,
        inputs: NDList,
        training: Boolean,
        params: PairList<String, Any>?
    ): NDList {
        var X = inputs.head()
        // Embedding indices must be integer type
        X = X.toType(DataType.INT64, false)
        // The output `X` shape: (`batchSize`, `numSteps`, `embedSize`)
        X = embedding.forward(ps, NDList(X), training, params).head()
        // In RNN models, the first axis corresponds to time steps
        X = X.swapAxes(0, 1)
        return rnn.forward(ps, NDList(X), training)
    }
}

The returned variables of recurrent layers
have been explained in :numref:`sec_rnn-concise`.
Let us still use a concrete example
to illustrate the above encoder implementation.
Below
we instantiate a two-layer GRU encoder
whose number of hidden units is 16.
Given
a minibatch of sequence inputs `X`
(batch size: 4, number of time steps: 7),
the hidden states of the last layer
at all the time steps
(`output` return by the encoder's recurrent layers)
are a tensor
of shape
(number of time steps, batch size, number of hidden units).


In [56]:
    var encoder = Seq2SeqEncoder(10, 8, 16, 2, 0.0f)
    var X = manager.zeros(Shape(4, 7), DataType.INT64)
    encoder.initialize(manager, DataType.FLOAT32, X.shape)
    var outputState = encoder.forward(ps, NDList(X), false)
    var output = outputState.head()

    println(output.shape)

(7, 4, 16)


Since a GRU is employed here,
the shape of the multilayer hidden states
at the final time step
is
(number of hidden layers, batch size, number of hidden units).
If an LSTM is used,
memory cell information will also be contained in `state`.


In [57]:
    var state = outputState.subNDList(1)
    println(state.size)
    println(state.head().shape)

1
(2, 4, 16)


## Decoder
:label:`sec_seq2seq_decoder`

As we just mentioned,
the context variable $\mathbf{c}$ of the encoder's output encodes the entire input sequence $x_1, \ldots, x_T$. Given the output sequence $y_1, y_2, \ldots, y_{T'}$ from the training dataset,
for each time step $t'$
(the symbol differs from the time step $t$ of input sequences or encoders),
the probability of the decoder output $y_{t'}$
is conditional
on the previous output subsequence
$y_1, \ldots, y_{t'-1}$ and
the context variable $\mathbf{c}$, i.e., $P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \mathbf{c})$.

To model this conditional probability on sequences,
we can use another RNN as the decoder.
At any time step $t^\prime$ on the output sequence,
the RNN takes the output $y_{t^\prime-1}$ from the previous time step
and the context variable $\mathbf{c}$ as its input,
then transforms
them and
the previous hidden state $\mathbf{s}_{t^\prime-1}$
into the
hidden state $\mathbf{s}_{t^\prime}$ at the current time step.
As a result, we can use a function $g$ to express the transformation of the decoder's hidden layer:

$$\mathbf{s}_{t^\prime} = g(y_{t^\prime-1}, \mathbf{c}, \mathbf{s}_{t^\prime-1}).$$
:eqlabel:`eq_seq2seq_s_t`

After obtaining the hidden state of the decoder,
we can use an output layer and the softmax operation to compute the conditional probability distribution
$P(y_{t^\prime} \mid y_1, \ldots, y_{t^\prime-1}, \mathbf{c})$ for the output at time step $t^\prime$.

Following :numref:`fig_seq2seq`,
when implementing the decoder as follows,
we directly use the hidden state at the final time step
of the encoder
to initialize the hidden state of the decoder.
This requires that the RNN encoder and the RNN decoder have the same number of layers and hidden units.
To further incorporate the encoded input sequence information,
the context variable is concatenated
with the decoder input at all the time steps.
To predict the probability distribution of the output token,
a fully-connected layer is used to transform
the hidden state at the final layer of the RNN decoder.


In [58]:
class Seq2SeqDecoder(vocabSize: Int, embedSize: Int, numHiddens: Int, numLayers: Int, dropout: Float) : Decoder() {
    private val embedding: TrainableWordEmbedding
    private val rnn: GRU
    private val dense: Linear

    /* The RNN decoder for sequence to sequence learning. */
    init {
        val list: List<String> = (0 until vocabSize).map { it.toString() }
        val vocab: Vocabulary = DefaultVocabulary(list)
        embedding = TrainableWordEmbedding.builder()
            .optNumEmbeddings(vocabSize)
            .setEmbeddingSize(embedSize)
            .setVocabulary(vocab)
            .build()
        addChildBlock("embedding", embedding)
        rnn = GRU.builder()
            .setNumLayers(numLayers)
            .setStateSize(numHiddens)
            .optReturnState(true)
            .optBatchFirst(false)
            .optDropRate(dropout)
            .build()
        addChildBlock("rnn", rnn)
        dense = Linear.builder().setUnits(vocabSize.toLong()).build()
        addChildBlock("dense", dense)
    }

    /** {@inheritDoc}  */
    override fun initializeChildBlocks(manager: NDManager, dataType: DataType, vararg inputShapes: Shape) {
        embedding.initialize(manager, dataType, inputShapes[0])
        manager.newSubManager().use { sub ->
            var shape: Shape? = embedding.getOutputShapes(arrayOf(inputShapes[0]))[0]
            val nd = sub.zeros(shape, dataType).swapAxes(0, 1)
            val state = sub.zeros(inputShapes[1], dataType)
            var context = state[NDIndex(-1)]
            context = context.broadcast(
                Shape(
                    nd.shape.head(),
                    context.shape.head(),
                    context.shape[1]
                )
            )
            // Broadcast `context` so it has the same `numSteps` as `X`
            val xAndContext = NDArrays.concat(NDList(nd, context), 2)
            rnn.initialize(manager, dataType, xAndContext.shape)
            shape = rnn.getOutputShapes(arrayOf(xAndContext.shape))[0]
            dense.initialize(manager, dataType, shape)
        }
    }

    override fun getOutputShapes(inputShapes: Array<Shape>): Array<Shape> {
        val embedShape = embedding.getOutputShapes(arrayOf(inputShapes[0]))[0]
        val xSwapped = Shape(embedShape[1], embedShape[0], embedShape[2])
        val stateShape = inputShapes[1]
        val xAndContextShape = Shape(xSwapped[0], xSwapped[1], xSwapped[2] + stateShape[2])
        val rnnShapes = rnn.getOutputShapes(arrayOf(xAndContextShape))
        val denseShape = dense.getOutputShapes(arrayOf(rnnShapes[0]))[0]
        val outShape = Shape(denseShape[1], denseShape[0], denseShape[2])
        return arrayOf(outShape, rnnShapes[1])
    }


    override fun initState(encOutputs: NDList): NDList {
        return NDList(encOutputs[1])
    }

    override fun forwardInternal(
        parameterStore: ParameterStore?,
        inputs: NDList,
        training: Boolean,
        params: PairList<String, Any>?
    ): NDList {
        var X = inputs.head()
        // Embedding indices must be integer type
        X = X.toType(DataType.INT64, false)
        var state = inputs[1]
        X = embedding
            .forward(parameterStore, NDList(X), training, params)
            .head()
            .swapAxes(0, 1)
        var context = state[NDIndex(-1)]
        // Broadcast `context` so it has the same `numSteps` as `X`
        context = context.broadcast(
            Shape(
                X.shape.head(),
                context.shape.head(),
                context.shape[1]
            )
        )
        val xAndContext = NDArrays.concat(NDList(X, context), 2)
        val rnnOutput = rnn.forward(parameterStore, NDList(xAndContext, state), training)
        var output = rnnOutput.head()
        state = rnnOutput[1]
        output = dense.forward(parameterStore, NDList(output), training)
            .head()
            .swapAxes(0, 1)
        return NDList(output, state)
    }
}


To illustrate the implemented decoder,
below we instantiate it with the same hyperparameters from the aforementioned encoder.
As we can see, the output shape of the decoder becomes (batch size, number of time steps, vocabulary size),
where the last dimension of the tensor stores the predicted token distribution.


In [59]:
    var decoder = Seq2SeqDecoder(10, 8, 16, 2, 0f)
    state = decoder.initState(outputState)
    val input = NDList(X).addAll(state)
    decoder.initialize(manager, DataType.FLOAT32, *input.shapes)
    outputState = decoder.forward(ps, input, false)

    output = outputState.head()
    println(output.shape)

    state = outputState.subNDList(1)
    println(state.size)
    println(state.head().shape)

(4, 7, 10)
1
(2, 4, 16)


To summarize,
the layers in the above RNN encoder-decoder model are illustrated in :numref:`fig_seq2seq_details`.

![Layers in an RNN encoder-decoder model.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/seq2seq-details.svg)
:label:`fig_seq2seq_details`

## Loss Function

At each time step, the decoder
predicts a probability distribution for the output tokens.
Similar to language modeling,
we can apply softmax to obtain the distribution
and calculate the cross-entropy loss for optimization.
Recall :numref:`sec_machine_translation`
that the special padding tokens
are appended to the end of sequences
so sequences of varying lengths
can be efficiently loaded
in minibatches of the same shape.
However,
prediction of padding tokens
should be excluded from loss calculations.

To this end,
we can use the following
`sequenceMask` function
to mask irrelevant entries with zero values
so later
multiplication of any irrelevant prediction
with zero equals to zero.
For example,
if the valid length of two sequences
excluding padding tokens
are one and two, respectively,
the remaining entries after
the first one
and the first two entries are cleared to zeros.


In [60]:
fun sequenceMask(X: NDArray, validLen: NDArray, value: Float = 0f): NDArray {
    val maxLen = X.shape.get(1)
    val valid = if (validLen.shape.dimension() == 1) validLen.reshape(-1, 1) else validLen
    var mask = X.manager.arange(maxLen.toInt()).expandDims(0).lt(valid)
    var maskF = mask.toType(X.dataType, false)
    while (maskF.shape.dimension() < X.shape.dimension()) {
        maskF = maskF.expandDims(-1)
    }
    maskF = maskF.broadcast(X.shape)
    val one = maskF.onesLike()
    val valueNd = X.manager.full(Shape(1), value).toType(X.dataType, false)
    return X.mul(maskF).add(one.sub(maskF).mul(valueNd))
}

    X = manager.create(arrayOf(intArrayOf(1, 2, 3), intArrayOf(4, 5, 6)))
    println(sequenceMask(X, manager.create(intArrayOf(1, 2))))

ND: (2, 3) gpu(0) int32
Check the "Development Guideline"->Debug to enable array display.



We can also mask all the entries across the last
few axes.
If you like, you may even specify
to replace such entries with a non-zero value.


In [61]:
fun sequenceMask(X: NDArray, validLen: NDArray, value: Float = 0f): NDArray {
    val maxLen = X.shape.get(1)
    val valid = if (validLen.shape.dimension() == 1) validLen.reshape(-1, 1) else validLen
    var mask = X.manager.arange(maxLen.toInt()).expandDims(0).lt(valid)
    var maskF = mask.toType(X.dataType, false)
    while (maskF.shape.dimension() < X.shape.dimension()) {
        maskF = maskF.expandDims(-1)
    }
    maskF = maskF.broadcast(X.shape)
    val one = maskF.onesLike()
    val valueNd = X.manager.full(Shape(1), value).toType(X.dataType, false)
    return X.mul(maskF).add(one.sub(maskF).mul(valueNd))
}

    X = manager.ones(Shape(2, 3, 4))
    println(sequenceMask(X, manager.create(intArrayOf(1, 2)), -1f))

ND: (2, 3, 4) gpu(0) float32
Check the "Development Guideline"->Debug to enable array display.



Now we can extend the softmax cross-entropy loss
to allow the masking of irrelevant predictions.
Initially,
masks for all the predicted tokens are set to one.
Once the valid length is given,
the mask corresponding to any padding token
will be cleared to zero.
In the end,
the loss for all the tokens
will be multipled by the mask to filter out
irrelevant predictions of padding tokens in the loss.


In [62]:
class MaskedSoftmaxCELoss : Loss("masked_softmax_ce") {

    /* The softmax cross-entropy loss with masks. */
    override fun evaluate(labels: NDList, predictions: NDList): NDArray {
        val y = labels.head().toType(DataType.INT64, false)
        val validLen = labels[1]
        val yHat = predictions.head()
        val logProb = yHat.logSoftmax(2)
        val yOneHot = y.oneHot(yHat.shape.get(2).toInt()).toType(yHat.dataType, false)
        val l = logProb.mul(yOneHot).sum(intArrayOf(2)).neg()
        val weights = sequenceMask(y.onesLike(), validLen)
        return l.mul(weights).mean(intArrayOf(1))
    }
}


For a sanity check, we can create three identical sequences.
Then we can
specify that the valid lengths of these sequences
are 4, 2, and 0, respectively.
As a result,
the loss of the first sequence
should be twice as large as that of the second sequence,
while the third sequence should have a zero loss.


In [63]:
    val loss: Loss = MaskedSoftmaxCELoss()
    val labels = NDList(manager.ones(Shape(3, 4)))
    labels.add(manager.create(intArrayOf(4, 2, 0)))
    val predictions = NDList(manager.ones(Shape(3, 4, 10)))
    println(loss.evaluate(labels, predictions))

ND: (3) gpu(0) float32
Check the "Development Guideline"->Debug to enable array display.



## Training
:label:`sec_seq2seq_training`

In the following training loop,
we concatenate the special beginning-of-sequence token
and the original output sequence excluding the final token as
the input to the decoder, as shown in :numref:`fig_seq2seq`.
This is called *teacher forcing* because
the original output sequence (token labels) is fed into the decoder.
Alternatively,
we could also feed the *predicted* token
from the previous time step
as the current input to the decoder.


In [64]:
    fun trainSeq2Seq(
        net: EncoderDecoder,
        dataset: ArrayDataset,
        lr: Float,
        numEpochs: Int,
        tgtVocab: Vocab,
        device: Device
    ) {
        val loss: Loss = MaskedSoftmaxCELoss()
        val lrt: Tracker = Tracker.fixed(lr)
        val adam: Optimizer = Optimizer.adam().optLearningRateTracker(lrt).build()
        val config: DefaultTrainingConfig = DefaultTrainingConfig(loss)
            .optOptimizer(adam) // Optimizer (loss function)
            .optInitializer(XavierInitializer(), "")
        val model: ai.djl.Model = ai.djl.Model.newInstance("")
        model.block = net
        val trainer: ai.djl.training.Trainer = model.newTrainer(config)
//    val animator = Animator()
        var watch: StopWatch
        var metric: Accumulator
        var lossValue = 0.0
        var speed = 0.0
        for (epoch in 1..numEpochs) {
            watch = StopWatch()
            metric = Accumulator(2) // Sum of training loss, no. of tokens
            manager.newSubManager(device).use { childManager ->
                // Iterate over dataset
                for (batch in dataset.getData(childManager)) {
                    val X: NDArray = batch.data.get(0)
                    val lenX: NDArray = batch.data.get(1)
                    val Y: NDArray = batch.labels.get(0)
                    val lenY: NDArray = batch.labels.get(1)
                    val bos: NDArray = childManager
                        .full(Shape(Y.shape[0]), tgtVocab.getIdx("<bos>"))
                        .reshape(-1, 1)
                    val decInput: NDArray = NDArrays.concat(
                        NDList(bos, Y.get(NDIndex(":, :-1"))),
                        1
                    ) // Teacher forcing
                    Engine.getInstance().newGradientCollector().use { gc ->
                        val yHat: NDArray = net.forward(
                            ParameterStore(manager, false),
                            NDList(X, decInput, lenX),
                            true
                        )
                            .get(0)
                        val l = loss.evaluate(NDList(Y, lenY), NDList(yHat))
                        gc.backward(l)
                        metric.add(floatArrayOf(l.sum().getFloat(), lenY.sum().getLong().toFloat()))
                    }
                    TrainingChapter9.gradClipping(net, 1, childManager)
                    // Update parameters
                    trainer.step()
                }
            }
            lossValue = metric.get(0).toDouble() / metric.get(1)
            speed = metric.get(1) / watch.stop()
            if ((epoch + 1) % 10 == 0) {
//            animator.add(epoch + 1, lossValue.toFloat(), "loss")
//            animator.show()
                println("${epoch + 1} : $lossValue")
            }
        }
        println(
            "loss: %.3f, %.1f tokens/sec on %s%n".format(lossValue, speed, device.toString())
        )
    }


Now we can create and train an RNN encoder-decoder model
for sequence to sequence learning on the machine translation dataset.


In [ ]:
    val embedSize = 32
    val numHiddens = 32
    val numLayers = 2
    val batchSize = 64
    val numSteps = 10
    val numEpochs = Integer.getInteger("MAX_EPOCH", 300)

    val dropout = 0.1f
    val lr = 0.005f
    val device = manager.device

    val dataNMT = NMT.loadDataNMT(batchSize, numSteps, 600)
    val dataset: ArrayDataset = dataNMT.first
    val srcVocab: Vocab = dataNMT.second.first
    val tgtVocab: Vocab = dataNMT.second.second

    encoder = Seq2SeqEncoder(srcVocab.length(), embedSize, numHiddens, numLayers, dropout)
    decoder = Seq2SeqDecoder(tgtVocab.length(), embedSize, numHiddens, numLayers, dropout)

    val net = EncoderDecoder(encoder, decoder)
    trainSeq2Seq(net, dataset, lr, numEpochs, tgtVocab, device)


10 : 0.1763478008511966
20 : 0.1077410785385306
30 : 0.07366815452170344
40 : 0.05429127590715393
50 : 0.04251001351276564
60 : 0.03486537820755239
70 : 0.03054870521398287
80 : 0.028475260368626402
90 : 0.025670417506410494


## Prediction

To predict the output sequence
token by token,
at each decoder time step
the predicted token from the previous
time step is fed into the decoder as an input.
Similar to training,
at the initial time step
the beginning-of-sequence ("&lt;bos&gt;") token
is fed into the decoder.
This prediction process
is illustrated in :numref:`fig_seq2seq_predict`.
When the end-of-sequence ("&lt;eos&gt;") token is predicted,
the prediction of the output sequence is complete.


![Predicting the output sequence token by token using an RNN encoder-decoder.](https://github.com/d2l-ai/d2l-en-colab/blob/master/img/seq2seq-predict.svg?raw=1)
:label:`fig_seq2seq_predict`

We will introduce different
strategies for sequence generation in
:numref:`sec_beam-search`.


In [66]:
/* Predict for sequence to sequence. */
    fun predictSeq2Seq(
        net: EncoderDecoder,
        srcSentence: String,
        srcVocab: Vocab,
        tgtVocab: Vocab,
        numSteps: Int,
        device: Device,
        saveAttentionWeights: Boolean
    ): Pair<String, List<NDArray?>> {
        val srcTokens: List<Int> =
            srcVocab.getIdxs(
                srcSentence.lowercase(Locale.getDefault()).split(" ".toRegex()).dropLastWhile { it.isEmpty() }
            ) + listOf(srcVocab.getIdx("<eos>"))
        val encValidLen = manager.create(srcTokens.size)
        val truncateSrcTokens = NMT.truncatePad(srcTokens, numSteps, srcVocab.getIdx("<pad>"))
        // Add the batch axis
        val encX = manager.create(truncateSrcTokens.toIntArray()).expandDims(0)
        val encOutputs = net.encoder.forward(ParameterStore(manager, false), NDList(encX, encValidLen), false)
        var decState = net.decoder.initState(encOutputs.addAll(NDList(encValidLen)))
        // Add the batch axis
        var decX = manager.create(intArrayOf(tgtVocab.getIdx("<bos>"))).expandDims(0)
        val outputSeq: MutableList<Int> = mutableListOf()
        val attentionWeightSeq: MutableList<NDArray?> = mutableListOf()
        for (i in 0 until numSteps) {
            val output = net.decoder.forward(
                ParameterStore(manager, false),
                NDList(decX).addAll(decState),
                false
            )
            val Y = output[0]
            decState = output.subNDList(1)
            // We use the token with the highest prediction likelihood as the input
            // of the decoder at the next time step
            decX = Y.argMax(2)
            val pred = decX.squeeze(0).getLong().toInt()
            // Save attention weights (to be covered later)
            if (saveAttentionWeights) {
                attentionWeightSeq.add(net.decoder.attentionWeights)
            }
            // Once the end-of-sequence token is predicted, the generation of the
            // output sequence is complete
            if (pred == tgtVocab.getIdx("<eos>")) {
                break
            }
            outputSeq.add(pred)
        }
        val outputString: String = " " + tgtVocab.toTokens(outputSeq)
        return Pair(outputString, attentionWeightSeq.toList())
    }

## Evaluation of Predicted Sequences

We can evaluate a predicted sequence
by comparing it with the
label sequence (the ground-truth).
BLEU (Bilingual Evaluation Understudy),
though originally proposed for evaluating
machine translation results :cite:`Papineni.Roukos.Ward.ea.2002`,
has been extensively used in measuring
the quality of output sequences for different applications.
In principle, for any $n$-grams in the predicted sequence,
BLEU evaluates whether this $n$-grams appears
in the label sequence.

Denote by $p_n$
the precision of $n$-grams,
which is
the ratio of
the number of matched $n$-grams in
the predicted and label sequences
to
the number of $n$-grams in the predicted sequence.
To explain,
given a label sequence $A$, $B$, $C$, $D$, $E$, $F$,
and a predicted sequence $A$, $B$, $B$, $C$, $D$,
we have $p_1 = 4/5$,  $p_2 = 3/4$, $p_3 = 1/3$, and $p_4 = 0$.
Besides,
let $\mathrm{len}_{\text{label}}$ and $\mathrm{len}_{\text{pred}}$
be
the numbers of tokens in the label sequence and the predicted sequence, respectively.
Then, BLEU is defined as

$$ \exp\left(\min\left(0, 1 - \frac{\mathrm{len}_{\text{label}}}{\mathrm{len}_{\text{pred}}}\right)\right) \prod_{n=1}^k p_n^{1/2^n},$$
:eqlabel:`eq_bleu`

where $k$ is the longest $n$-grams for matching.

Based on the definition of BLEU in :eqref:`eq_bleu`,
whenever the predicted sequence is the same as the label sequence, BLEU is 1.
Moreover,
since matching longer $n$-grams is more difficult,
BLEU assigns a greater weight
to a longer $n$-gram precision.
Specifically, when $p_n$ is fixed,
$p_n^{1/2^n}$ increases as $n$ grows (the original paper uses $p_n^{1/n}$).
Furthermore,
since
predicting shorter sequences
tends to obtain a higher $p_n$ value,
the coefficient before the multiplication term in :eqref:`eq_bleu`
penalizes shorter predicted sequences.
For example, when $k=2$,
given the label sequence $A$, $B$, $C$, $D$, $E$, $F$ and the predicted sequence $A$, $B$,
although $p_1 = p_2 = 1$, the penalty factor $\exp(1-6/2) \approx 0.14$ lowers the BLEU.

We implement the BLEU measure as follows.


In [67]:
/* Compute the BLEU. */
    fun bleu(predSeq: String, labelSeq: String, k: Int): Double {
        fun tokenize(seq: String): List<String> {
            return seq
                .replace("[", " ")
                .replace("]", " ")
                .replace(",", " ")
                .trim()
                .split(" ".toRegex())
                .filter { it.isNotEmpty() }
        }
        val predTokens = tokenize(predSeq)
        val labelTokens = tokenize(labelSeq)
        val lenPred = predTokens.size
        val lenLabel = labelTokens.size
        if (lenPred == 0) {
            return 0.0
        }
        var score = Math.exp(Math.min(0.0, 1.0 - lenLabel.toDouble() / lenPred.toDouble()))
        for (n in 1 until k + 1) {
            val maxPredSub = lenPred - n + 1
            if (maxPredSub <= 0) {
                break
            }
            var numMatches = 0.0
            val labelSubs: MutableMap<String, Int> = mutableMapOf()
            for (i in 0 until lenLabel - n + 1) {
                val key: String = " " + labelTokens.subList(i, i + n)
                labelSubs[key] = labelSubs.getOrDefault(key, 0) + 1
            }
            for (i in 0 until maxPredSub) {
                val key: String = " " + predTokens.subList(i, i + n)
                if (labelSubs.getOrDefault(key, 0) > 0) {
                    numMatches += 1.0
                    labelSubs[key] = labelSubs.getOrDefault(key, 0) - 1
                }
            }
            val precision = numMatches / maxPredSub.toDouble()
            score *= Math.pow(precision, Math.pow(0.5, n.toDouble()))
        }
        return score
    }


In the end,
we use the trained RNN encoder-decoder
to translate a few English sentences into French
and compute the BLEU of the results.


In [69]:
    val engs = arrayOf("go .", "i lost .", "he\'s calm .", "i\'m home .")
    val fras = arrayOf("va !", "j\'ai perdu .", "il est calme .", "je suis chez moi .")
    for (i in engs.indices) {
        val pair = predictSeq2Seq(net, engs[i], srcVocab, tgtVocab, numSteps, device, false)
        val translation: String = pair.first
        val attentionWeightSeq = pair.second
        println("%s => %s, bleu %.3f".format(engs[i], translation, bleu(translation, fras[i], 2)))
    }

go . =>  [va, un, pigé, .], bleu 0.000
i lost . =>  [j'ai, compris, .], bleu 0.000
he's calm . =>  [<unk>, <unk>, .], bleu 0.000
i'm home . =>  [je, suis, chez, moi, tombé, emporté, .], bleu 0.711


## Summary

* Following the design of the encoder-decoder architecture, we can use two RNNs to design a model for sequence to sequence learning.
* When implementing the encoder and the decoder, we can use multilayer RNNs.
* We can use masks to filter out irrelevant computations, such as when calculating the loss.
* In encoder-decoder training, the teacher forcing approach feeds original output sequences (in contrast to predictions) into the decoder.
* BLEU is a popular measure for evaluating output sequences by matching $n$-grams between the predicted sequence and the label sequence.


## Exercises

1. Can you adjust the hyperparameters to improve the translation results?
1. Rerun the experiment without using masks in the loss calculation. What results do you observe? Why?
1. If the encoder and the decoder differ in the number of layers or the number of hidden units, how can we initialize the hidden state of the decoder?
1. In training, replace teacher forcing with feeding the prediction at the previous time step into the decoder. How does this influence the performance?
1. Rerun the experiment by replacing GRU with LSTM.
1. Are there any other ways to design the output layer of the decoder?
